In [ ]:
from importlib import reload
import numpy as np
from keras.models import Model
from keras.layers import Dense, Conv2D, MaxPooling2D, Dropout
from keras.layers import Activation, Flatten, ZeroPadding2D, GlobalAveragePooling2D
from keras.optimizers import SGD, Adam, rmsprop
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import TensorBoard
from keras.models import Sequential

In [ ]:
train_dir = '/src/dataset/kaggle/dog_cat_redux/train'
val_dir = '/src/dataset/kaggle/dog_cat_redux/valid'
num_label = 2
batch_size = 128
num_of_epochs = 10
num_training = 23000
num_val = 2000
im_width, im_height = 299, 299

In [ ]:
# Create model
model = Sequential()
model.add(Conv2D(16, (3, 3), input_shape=(im_width, im_height, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D())
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dense(num_label))
model.add(Activation('softmax'))

opt = rmsprop(lr=0.0001, decay=1e-6)
model.compile(optimizer=opt,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
train_datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    rescale=1/255,
    horizontal_flip=True,
    vertical_flip=False
)

val_datagen = ImageDataGenerator(
    rescale=1/255,
)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(im_width, im_height),
    class_mode="categorical",
    batch_size=batch_size,
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(im_width, im_height),
    class_mode="categorical",
    batch_size=batch_size,
)

In [ ]:
# Plot learning curve on tensor board
tensor_board = TensorBoard(log_dir='/src/tensorboard', batch_size=batch_size)

# Fit the model on the batches generated by datagen.flow().
model.fit_generator(train_generator, 
                    epochs = num_of_epochs,
                    validation_data=val_generator,
                    callbacks = [tensor_board])